In [0]:
"""
https://www.youtube.com/watch?v=J9wwR4huimI

This is somewhat similar to SCD type 1
Common ids should be excluded from output
1,2 -- common so exclude
4 is mismatch
3 is new in source
5 is new in target
"""

from pyspark.sql.functions import *
from pyspark.sql.window import *
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

source = spark.createDataFrame([
    (1,'A'),
    (2,'B'),
    (3,'C'),
    (4,'D')
], ["s_id", "s_name"])

target = spark.createDataFrame([
    (1,'A'),
    (2,'B'),
    (4,'X'),
    (5,'F')
], ["t_id", "t_name"]) 

source.show()
target.show()

+----+------+
|s_id|s_name|
+----+------+
|   1|     A|
|   2|     B|
|   3|     C|
|   4|     D|
+----+------+

+----+------+
|t_id|t_name|
+----+------+
|   1|     A|
|   2|     B|
|   4|     X|
|   5|     F|
+----+------+



In [0]:
source.join(target, source["s_id"]==target["t_id"], "outer") \
    .withColumn("comment_", when(col("s_name")==col("t_name"), "exclude").when(col("t_name").isNull(), "new in source").when(col("s_name").isNull(), "new in target").when(col("s_name")!=col("t_name"), "mismatched")) \
    .withColumn("s_id", when(col("s_id").isNull(), col("t_id")).otherwise(col("s_id")))\
    .select("s_id", "comment_") \
    .filter(col("comment_") != "exclude") \
    .show()

+----+-------------+
|s_id|     comment_|
+----+-------------+
|   3|new in source|
|   4|   mismatched|
|   5|new in target|
+----+-------------+

